# quantum_machine_learning.myQML.QCBM

**class quantum_machine_learning.myQML.QCBM(n_qubits, basis, n_blocks, n_shots, device, sigma_list_kernel, ansatz_mode=0, execution_mode=0, dimension=None)**    

QCBM is a generative model that encodes the probability distribution of classical data as a variational quantum circuit, and it is based on Born's postulate of Quantum Mechanics, which states that the probabiliy of obtaining the bitstring x as a result of measuring the quantum state $|\psi_\theta \rangle$ is $|\langle x | \psi_\theta \rangle|^2$. In contrast to other methods such as Restricted Boltzmann machines, QCBM samples efficiently, as it just requires executing the quantum circuit. 

Regarding the training process, the variatioanl circuit is initialized with random parameters, and for each iteration it produces a specified number of samples and a loss function is evaluated, which measures the distance between the target distribution and the model distribution. The QCBM method employs the squared mean discrepancy loss with a Radial Basis Function (RBF) kernel. Similarly to supervised QNNs, the gradient f the loss function with respect to the variational parameters can be evaluated exactly via the Parameter Shift Rule. Thus, we can use gradient-based optimizers such as Adam or L-BFGS-B.

After the optimal parameters are found and training is finished, we will generate samples from the circuit and analyse their validity. Apart from the precision metric, which measure the rate of generating valid patterns, there are other metrics that measure generalzation, which refers to not only considering the validity criterium but also whether the generated sample was seen in training. However, since our dataset is very small, it does not make sense to study its generalizability.


**Parameters:**
- **n_qubits** (int): Number of qubits employed for the parametrized quantum circuit. $2^{n_\text{qubits}}$ must be equal or greater than the number of possible states in the dataset.
- **basis** (numpy.ndarray): Array of shape $(2^{n_{\text{qubits}}}, \sqrt{n_{\text{qubits}}}, \sqrt{n_{\text{qubits}}})$
, assuming the dataset is made of squared images of N x N = n_qubits pixels.
- **n_blocks** (int): Number of blocks in the variational quantum layer, where each of the blocks contains rotational gates and CNOTs (or other entangling two-qubit gates).
- **n_shots** (int): Number of executions of the corresponding quantum circuit to estimate a probability or the expectation value of an observable. If n_shots = None, the estimation of the quantum simulator has no shot noise.
- **optimizer_name** (str): Label of the optimizer used for the training of the classical neural network. Only 'Adam' optimizer is implemented currently.
- **device** (str): Can be either 'myQLM' or 'Qaptiva', depending if you want to use the Qaptiva plugin instead of your own laptop for the quantum circuit simulations.
- **sigma_list_kernel** (list): List of the standard deviations of the gaussians in the multi radial basis function (RBF) kernel, used for the two sample test loss.
- **ansatz_mode** (int): Given that there are many possible ansatz choices for parametrized quantum circuits, this variables selects the one chosen. Currently, only one ansatz is implemented, accessed using ansatz_mode = 0.
- **execution_mode** (int): If equals to 0, the quantum circuit is executed with no shot noise in myQLM and then samples are obtained using numpy.random; and if equals to 1, shot noise in myQLM is implemented. 
- **dimension** : Shape of the images. The products of the dimensions must be equal to the number of qubits required for the parametrized quantum circuit.


**ansatz(params)**

This method defines the parametrized quantum circuit with given rotational gates angles/parameters, measuring in the computational basis n_shots times, obtaining n_shots samples.

**Parameters:**
- **params** (numpy.ndarray): Array of n_params = 3 x n_qubits x n_blocks elements with the angles of the rotational gates of the quantum circuit. These parameters are varied to minimize the loss and learn the target probability distribution.

**Returns**: samples (numpy.ndarray) - n_shots samples obtained by the quantum measurement in the computational basis, and circuit (qat.core.circuit.Circuit) - the compiled quantum circuit in myQLM.


**estimate_probs(params)**

This method calls the ansatz method, and with the returned samples, estimates the corresponding probability distribution. 

**Parameters:**
- **params** (numpy.ndarray): Array containing the angles of all rotational gates of the parametrized quantumcircuit. The number of elements is 3 x n_qubits x n_blocks using the Hardware Efficient Ansatz (HEA).

**Returns:** The estimated probability distribution, which approximates the target distribution.

**Return type:**  numpy.ndarray of $2^{n_\text{qubits}}$ elements.


**multi_rbf_kernel(x, y, sigma_list)**

Builds the kernel matrix for all combinations of basis elemens, using the multi radial basis function (RBF) kernel with the specified standard distributions, to obtain a measure of distance between basis elements.

**Parameters:**
- **x** (numpy.ndarray): Array of the basis elements of the dataset utilized.
- **y** (numpy.ndarray): Array of the basis elements of the dataset utilized.
- **sigma_list** (list): List of the standard deviations of the gaussians in the multi radial basis function (RBF) kernel, used for the two sample test loss.


**Returns:** The kernel matrix of dimension $2^{n_\text{qubits}}$ x $2^{n_\text{qubits}}$.

**Return type:** numpy.ndarray



**kernel_expectation(px, py, kernel_matrix)**

Given the probability distribution vectors px and py, which can be the target distribution or the quantum circuit distribution, it computes their inner product using the kernel matrix computed previously. Hence, it returns a number (float) which is used to compute the MMD loss function.

**Parameters:**
- **px** (numpy.ndarray): Probability distribution vector (probability density of each state)
- **py** (numy.ndarray): Probability distribution vector (probability density of each state)
- **kernel_matrix** (numpy.ndarray): Kernel matrix using the multi RBF kernel.

**Returns:** The result of the inner product of px and py using the kernel matrix.

**Return type:** float



**loss_function(params, target_probs, kernel_matrix)**

After estimating the probabilities of each state resulting from thr parametrized quantum circuit using the estimate_probs method, it calls the kernel_expectation method with target_probs and p_circuit as inputs (and also with repeated inputs, e.g. px=py=target_probs and px=py=p_circuit), and then calculated the maximum mean discrepancy (MMD) loss. This measures how different the quantum circuit and target distributions are.

**Parameters:**
- **params** (numpy.ndarray): Array of n_params = 3 x n_qubits x n_blocks elements with the angles of the rotational gates of the quantum circuit. These parameters are varied to minimize the loss and learn the target probability distribution.
- **target_probs** (numpy.ndarray): Array with the probability densities of each state in the training set.
- **kernel_matrix** (numpy.ndarray): Kernel matrix using the multi RBF kernel.

**Returns:** The MMD loss between the quantum circuit and target distributions.

**Return type:** float



**gradient(theta, kernel_matrix, target_probs)**

This method calculated the gradient of the MMD loss functionwith respect to the angle parameters using the Parameter Shift Rule.

**Parameters:**
- **theta** (numpy.ndarray):Array of n_params = 3 x n_qubits x n_blocks elements with the angles of the rotational gates of the quantum circuit. These parameters are varied to minimize the loss and learn the target probability distribution. Same as params.
- **kernel_matrix** (numpy.ndarray): Kernel matrix using the multi RBF kernel.
- **target_probs** (numpy.ndarray): Array with the probability densities of each state in the training set.

**Returns:**  An array containing the gradient of the loss with respect to all the parameters.

**Return type:** numpy.ndarray



**fit(method="L-BFGS-B", learning_rate=0.1, tol=1e-5, max_iter=20, g_tol=1e-10, f_tol=0, x_tr=None, target_probs=None)**

This method optimizes the mean discrepancy loss (MMD) between the target probability distribution obtained from the training data and the quantum circuit model distribution. The optimizer employed is gradient-based and can be chosen to be L-BFGS-B or Adam.

**Parameters:**
- **method** (str): 'L-BFGS-B' or 'Adam'
- **learning_rate** (float): Step size multiplier of the update of parameters in Adam optimization.
- **tol** (float): Overall termination tolerance. If not None, this provides a global stopping condition for convergence (can combine with gtol and ftol)
- **max_iter** (int): Maximum number of iterations the optimizer is allowed to perform. Prevents infinite or excessively long optimization runs.
- **g_tol** (float): Gradient tolerance (for L-BFGS-B). Optimization stops when the gradient norm is smaller than this threshold, meaning the algorithm considers the parameters to be close enough to a stationary point (minimum).
- **f_tol** (float): Function tolerance (for L-BFGS-B). Optimization stops when the relative or absolute change in the loss function value between iterations is smaller than this threshold. Ensures we don’t waste time on negligible improvements.
- **x_tr** (numpy.ndarray): Training dataset
- **target_probs** (numpy.ndarray): Array with the probability densities of each state in the training set.

**Returns:** result (a scipy.optimize.OptimizeResult object with the loss in each epoch), tracking_cost (a list of the tracking cost in each epoch), and the optimization time (float). 



**plot_loss(tracking_cost)**
 
 Plots the MMD loss in each epoch.


**Parameters:**
- **tracking_cost** (list): List of the MMD loss in each epoch.

**Returns:** This method does not return any variable, but shows the figure in the notebook/terminal.



**plot_generated_distribution(samples)**

Plots the probability distribution of the generated samples of the optimized parametrized quantum circuit.

**Parameters:**
- **samples** (numpy.ndarray): Array of size n_samples x N x N generated samples (matrices of N x N = n_qubits pixels) after training is completed.

**Returns:** This method does not return any variable, but shows the figure in the notebook/terminal.


**generate_samples(n_samples)**

Executes the ansatz method with the optimized params n_shots = n_samples times, obtaining n_samples samples.


**Parameters:**
- **n_samples** (int): Number of generated samples.

**Returns:** samples_matrix (numpy.ndarray) - Array of size n_samples x N x N generated samples.



**plot_generated_samples(samples_matrix)**

This method plots the generated samples matrices and shows them in the notebook/terminal.

**Parameters:**
- **samples_matrix** (numpy.ndarray): Array of size n_samples x N x N generated samples (matrices of N x N = n_qubits pixels) after training is completed.

**Returns:** This method does not return any variable, but shows the figure in the notebook/terminal.



**calculate_metrics(samples_matrix, x_tr, validity_fn, n_sols, cost_fn)**

Calculates memorization and generalization metrics of the generated samples, such as cost, precision, fidelity, rate and coverage.

**Parameters:**
- **samples_matrix** (numpy.ndarray): Array of size n_samples x N x N generated samples (matrices of N x N = n_qubits pixels) after training is completed.
- **x_tr** (numpy.ndarray): Training dataset
- **validity_fn** (function): Function that returns true if the generated sample is valid (depends on the specific dataset)
- **n_sols** (int): Numbre of valid elements in the dataset
- **cost_fn** (function): Cost function to see how off a sample is from being valid

**Returns:** A dictionary with all the metrics.
